In [6]:
# https://github.com/iannellif/viralrank_centrality/blob/master/vrank.py
# https://physics.paperswithcode.com/paper/influencers-identification-in-complex

# https://github.com/asalavaty/python-influential/tree/main

# https://influential.erc.monash.edu/

import json
import numpy as np
import networkx as nx
from tqdm import tqdm

In [7]:
# Open the JSON file in read mode
json_file_path = "/home/saurabh/AAA/Convergent/Projects/social-media-data-analysis/data/final_data.json"
with open(json_file_path, "r") as json_file:
    data = json.load(json_file)

print(f"Total no. of node: {len(data)}")

# Initialize a directed graph for the Friendship Network
G = nx.DiGraph()

relational_data = []
for user in data:
    if len(user['followers']) or len(user['followings']):
        relational_data.append(user)

print(f"connected nodes: {len(relational_data)}")
data = list(relational_data)

# Add nodes (users) to the graph
for user in data:
    username = user['username']
    G.add_node(username)
    G.nodes[username]['metadata'] = user

# Add directed edges based on follower and followee relationships
for user in data:
    username = user['username']
    followings = user['followings']  # Assuming 'followings' contains user IDs being followed
    for following_user_id in followings:
        following_user = next((x for x in data if x['id'] == following_user_id), None)
        if following_user:
            following_username = following_user['username']
            G.add_edge(username, following_username)

Total no. of node: 300
connected nodes: 298


In [8]:
in_degree = dict(G.in_degree())
print(in_degree)

{'molly0xfff': 1, 'pluralistic': 3, 'jeffjarvis': 2, 'erictopol': 4, 'timnitGebru': 4, 'dangillmor': 0, 'ryanhoulihan': 0, 'TheConversationUS': 2, 'emilymbender': 1, 'racheltobac': 3, 'carnage4life': 2, 'tinker': 4, 'glennf': 1, 'Mer__edith': 3, 'parismarx': 2, 'Riedl': 2, 'openrightsgroup': 3, 'TechDesk': 2, 'DAIR': 5, 'mmitchell_ai': 7, 'davidpierce': 1, 'alex': 1, 'PCMag': 2, 'longreads': 3, 'thegradient': 1, 'EDPS': 2, 'Ailantd': 2, 'glynmoody': 2, 'ben': 5, 'garymarcus': 3, 'CultureDesk': 2, 'farhanasultana': 0, 'MattHodges': 7, 'LukaszOlejnik': 1, 'karenhao': 3, 'lightweight': 5, 'Iris': 5, 'janeadams': 6, 'adfichter': 1, 'forgejo': 1, 'annika': 1, 'olivia': 1, 'smach': 4, 'Forbes': 2, 'fgusmao': 3, 'jaredwhite': 2, 'philosophy': 1, 'deonandan': 2, 'revkin': 8, 'dweinberger': 7, 'docpop': 3, 'b9AcE': 2, 'reichenstein': 1, 'tyrell_turing': 1, 'Wikisteff': 4, 'sethlazar': 4, 'metin': 2, 'axbom': 2, 'jesper': 2, 'garry': 5, 'JorgeOMau': 1, 'dznz': 1, 'JL_Lycette': 6, 'omearabrian': 

In [9]:
out_degree = dict(G.out_degree())
print(out_degree)

{'molly0xfff': 0, 'pluralistic': 1, 'jeffjarvis': 9, 'erictopol': 0, 'timnitGebru': 0, 'dangillmor': 5, 'ryanhoulihan': 40, 'TheConversationUS': 2, 'emilymbender': 0, 'racheltobac': 0, 'carnage4life': 0, 'tinker': 0, 'glennf': 0, 'Mer__edith': 0, 'parismarx': 0, 'Riedl': 0, 'openrightsgroup': 0, 'TechDesk': 4, 'DAIR': 0, 'mmitchell_ai': 3, 'davidpierce': 1, 'alex': 1, 'PCMag': 0, 'longreads': 0, 'thegradient': 0, 'EDPS': 0, 'Ailantd': 0, 'glynmoody': 5, 'ben': 6, 'garymarcus': 1, 'CultureDesk': 1, 'farhanasultana': 4, 'MattHodges': 4, 'LukaszOlejnik': 3, 'karenhao': 0, 'lightweight': 1, 'Iris': 0, 'janeadams': 4, 'adfichter': 0, 'forgejo': 0, 'annika': 0, 'olivia': 0, 'smach': 1, 'Forbes': 0, 'fgusmao': 54, 'jaredwhite': 0, 'philosophy': 0, 'deonandan': 0, 'revkin': 0, 'dweinberger': 6, 'docpop': 2, 'b9AcE': 0, 'reichenstein': 0, 'tyrell_turing': 0, 'Wikisteff': 9, 'sethlazar': 3, 'metin': 0, 'axbom': 3, 'jesper': 0, 'garry': 0, 'JorgeOMau': 0, 'dznz': 0, 'JL_Lycette': 1, 'omearabrian'

In [12]:
import community as community_louvain

# Calculate the modularity of the graph
partition = community_louvain.best_partition(G)
modularity = community_louvain.modularity(partition, G)
print(modularity)

AttributeError: module 'community' has no attribute 'best_partition'

In [4]:
class ViralRank:
    
    def __init__(self,graph):
        self.graph = graph
       
        
    def value(self, inversetemp = 1e-4):
        """
        Compute the ViralRank centrality index from:
        F. Iannelli, M.S. Mariani, I.M. Sokolov 
        "Influencers identification in complex networks through reaction-diffusion dynamics"
        (https://arxiv.org/abs/1803.01212)
        Viralrank is defined as (note that the minus sign implies negative scores): 
        v(lambda) = - \sum_j (D_{ij} + D_{ji})/N
        where D_{ij} is the random-walk effective distance [Iannelli at al. Phys. Rev. E 95, 012313 (2017)] 
        from i to j and lambda is a parameter that plays the role of the inverse temperature (see next), 
        while N is the number of nodes.
        
        Parameters
        --------
                
            inversetemp : float
                numerical parameter (lambda) necessary for the computation of ViralRank centrality
                for metapopulation models. It is defined as:
                    lambda = log[ (beta-mu)/alpha ] - gamma
                where beta, mu and alpha are the infection, recovery and  
                diffusion rates, respectively, while gamma is the Euler constant.
                For contact networks its value is set equal to (approximately) zero corresponding
                to a high temperature expansion, while it can be specifically tuned 
                for metapopulation networks uisng the above definition.
                  
                
        Returns:
        --------
        
            nodes' ViralRank score : ndarray
                
        """           
        
        # assert nx.is_connected(self.graph), "The network has to be connected" 
        assert inversetemp > 0, "Negative temperature"
        A = np.asarray(nx.to_numpy_array(self.graph, dtype=float, weight=None).T)
        A = np.asarray(nx.to_numpy_array(self.graph, dtype=float, weight=None).T)
        # print("The graph is directed:", nx.is_directed(G))
        #N = g.number_of_nodes()
        s = A.sum(1)
        P = (A.T/s).T
        #print("sum over columns of P", P.sum(1))  
        s = A.sum(1)
        #assert np.all(s) > 0, "The network has to be connected"
        P = (A.T/s).T        
        assert np.all(np.isclose(P.sum(axis=1), 1, rtol=1e-10)), "Non valid transition matrix" 
        
        
        self.nodes = self.graph.number_of_nodes()
        targets = range(self.nodes)
               
        I = np.eye( self.nodes-1,self.nodes-1 )
        Z = np.ones( (self.nodes, self.nodes) )
        D = np.zeros( (self.nodes, self.nodes) )
        for j in tqdm(targets): 
            Pm = np.delete(P,j,0) 
            Pm = np.delete(Pm,j,1) 
            pm = P[:,j]  
            pm = np.delete(pm,j) 
            z = np.linalg.solve((np.exp(inversetemp)*I-Pm), pm)
            Z[:,j] = np.insert(z,j,1)
            D[:,j] = -np.log(Z[:,j])
        
        v = -1./self.nodes * (D.sum(1)+D.sum(0)) 
        
        return v  
    
    def rank(self, inversetemp = 1e-4):
        
        values = self.value()
        return np.argsort(values)

In [5]:
rank_result = ViralRank(G)

  0%|          | 0/297 [00:00<?, ?it/s]/tmp/ipykernel_77650/3290071297.py:68: RuntimeWarning: divide by zero encountered in log
  D[:,j] = -np.log(Z[:,j])
100%|██████████| 297/297 [00:02<00:00, 110.67it/s]


In [6]:
rank_result.rank()

array([  0, 200, 199, 198, 197, 196, 195, 201, 194, 192, 191, 190, 189,
       188, 187, 193, 202, 203, 204, 219, 218, 217, 216, 215, 214, 213,
       212, 211, 210, 209, 208, 207, 206, 205, 186, 220, 185, 183, 163,
       162, 161, 160, 159, 158, 164, 157, 155, 154, 153, 152, 151, 150,
       156, 165, 166, 167, 182, 181, 180, 179, 178, 177, 176, 175, 174,
       173, 172, 171, 170, 169, 168, 184, 221, 222, 223, 275, 274, 273,
       272, 271, 270, 276, 269, 267, 266, 265, 264, 263, 262, 268, 277,
       278, 279, 294, 293, 292, 291, 290, 289, 288, 287, 286, 285, 284,
       283, 282, 281, 280, 261, 260, 259, 258, 238, 237, 236, 235, 234,
       233, 232, 231, 230, 229, 228, 227, 226, 225, 224, 239, 149, 240,
       242, 257, 256, 255, 254, 253, 252, 251, 250, 249, 248, 247, 246,
       245, 244, 243, 241, 295, 148, 146,  51,  50,  49,  48,  47,  46,
        52,  45,  43,  42,  41,  40,  39,  38,  44,  53,  54,  55,  70,
        69,  68,  67,  66,  65,  64,  63,  62,  61,  60,  59,  5

In [253]:
for i,j in enumerate(G.nodes()):
    G.nodes[j]['rank'] = rank_nodes[i]

In [255]:
import influential

In [256]:
influential.ivi(G)

AttributeError: 'Graph' object has no attribute 'vs'

In [257]:
influential.collective_influence(G)

AttributeError: 'Graph' object has no attribute 'vs'